This file is for informational purposes only, the models are already created in "./models/"  
Do not run this file unless you want to create new models with modifications.

In [2]:
from keras import backend as K
from keras_preprocessing import sequence
from keras.layers import *
from keras.models import Model, Sequential
import numpy as np
import time
import os
import joblib
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import OneHotEncoder
from fastai.imports import *
from fastai.structured import *
from IPython.display import display

Using TensorFlow backend.
C:\Users\ohjia\anaconda3\envs\fastai2\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


Read in training and test data, returns:
* x: training set features
* y: training set labels
* test_x: test set features
* test_y: test set labels

In [3]:
train = np.load('dataset/train.npy', allow_pickle=True)

x = np.zeros((1801 * 185, 32))
y = np.zeros((1801 * 185))
n = 0
for i in range(1801):
    # pad the current song's bars
    for j in range(185):
        if j < len(train[i][1]) and 1 in train[i][1][j]:
            y[n] = train[i][1][j].index(1)
        else:
            y[n] = 24
            
        for k in range(32):
            if j < len(train[i][0]) and k < len(train[i][0][j]) and 1 in train[i][0][j][k]:
                x[n][k] = train[i][0][j][k].index(1)
            else:
                x[n][k] = 12
        n += 1

train = np.load('dataset/test.npy', allow_pickle=True)

test_x = np.zeros((train.shape[0] * 185, 32))
test_y = np.zeros((train.shape[0] * 185))

n = 0
for i in range(train.shape[0]):
    for j in range(185):
        if j < len(train[i][1]) and 1 in train[i][1][j]:
            test_y[n] = train[i][1][j].index(1)
        else:
            test_y[n] = 24
            
        for k in range(32):
            if j < len(train[i][0]) and k < len(train[i][0][j]) and 1 in train[i][0][j][k]:
                test_x[n][k] = train[i][0][j][k].index(1)
            else:
                test_x[n][k] = 12
        n += 1
rem = []
for i in range(len(x)):
    if all(x[i] == 12):
        rem.append(i)
rem_test = []
for i in range(len(test_x)):
    if all(test_x[i] == 12):
        rem_test.append(i)
x = np.delete(x, rem, 0)
y = np.delete(y, rem, 0)
test_x = np.delete(test_x, rem_test, 0)
test_y = np.delete(test_y, rem_test, 0)

## Random Forest Classifier

Model definition and training

In [5]:
m = RandomForestClassifier(n_estimators = 30, min_samples_leaf = 100, max_features = None, bootstrap = True, n_jobs = -1, random_state = 3244)
m.fit(x, y)

RandomForestClassifier(max_features=None, min_samples_leaf=100, n_estimators=30,
                       n_jobs=-1, random_state=3244)

Save model

In [19]:
joblib.dump(m, "models/RFC.pkl")

['models/RFC.pkl']

If want to load model

In [7]:
m = joblib.load("models/RFC.pkl")

Predict test y

In [8]:
y_pred = m.predict(test_x)
print(accuracy_score(y_pred, test_y))

0.4373272908146347


## LSTM

Onehot encode training y

In [6]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_onehot = onehot_encoder.fit_transform(integer_encoded)

Model definition and training

In [14]:
model = Sequential()
model.add(Embedding(input_dim = 24 + 1, output_dim = 50, input_length = 32))
model.add(LSTM(50))
model.add(Dense(16, activation = "relu"))
model.add(Dense(24, activation = "softmax"))

In [15]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy")
history2 = model.fit(train_x, train_y_onehot, epochs = 25, validation_data = (val_x, val_y_onehot), verbose = 1)

C:\Users\ohjia\anaconda3\envs\fastai2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
333185/333185 [==============================] - 196s 587us/step - loss: 0.5618
Epoch 2/10
333185/333185 [==============================] - 203s 608us/step - loss: 0.5675
Epoch 3/10
333185/333185 [==============================] - 197s 590us/step - loss: 0.4963
Epoch 4/10
333185/333185 [==============================] - 207s 620us/step - loss: 0.4410
Epoch 5/10
333185/333185 [==============================] - 170s 510us/step - loss: 0.4518
Epoch 6/10
333185/333185 [==============================] - 147s 441us/step - loss: 0.4680
Epoch 7/10
333185/333185 [==============================] - 149s 446us/step - loss: 0.4277
Epoch 8/10
333185/333185 [==============================] - 155s 465us/step - loss: 0.4205
Epoch 9/10
333185/333185 [==============================] - 168s 505us/step - loss: 0.4250
Epoch 10/10
333185/333185 [==============================] - 186s 560us/step - loss: 0.4259


Save Model

In [18]:
joblib.dump(model, "models/LSTM.pkl")

['models/LSTM.pkl']

If want to load model

In [4]:
model = joblib.load("models/LSTM.pkl")

C:\Users\ohjia\anaconda3\envs\fastai2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Onehot encode test y

In [7]:
test_integer_encoded = label_encoder.transform(test_y)
test_integer_encoded = test_integer_encoded.reshape(len(test_integer_encoded), 1)
test_y_onehot = onehot_encoder.transform(test_integer_encoded)

Predict test set

In [8]:
pred_y_onehot = model.predict(test_x)
pred_y = []
for cur_y in pred_y_onehot:
    inverted = label_encoder.inverse_transform([np.argmax(cur_y)])
    pred_y.append(inverted)
pred_y2 = []
for arr in pred_y:
    pred_y2.append(arr[0])
accuracy_score(test_y, pred_y2)

0.4654028959876202